## Behaviral Law Engine
- Stage: Cambrian
- Version: Pomoria

In [1]:
import logging 
logging.basicConfig(level=logging.CRITICAL)

In [2]:
import os
import itertools
import time
import copy
import random
import pickle
import numpy as np

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

# from tensorboardX import SummaryWriter

use_cuda = torch.cuda.is_available()
# use_cuda = False
print("use_cuda: {}".format(use_cuda))

use_cuda: True


In [4]:
from MorpheusInterpreter import *
from ProgramSpace import *

In [ ]:
torch.__version__

'1.0.0'

In [ ]:
class ProgramDataset(Dataset):
    def __init__(self, p_config=None, pn_program=None, p_generator=None, p_sourceps=None):
        self.config = p_config
        self.n_program = pn_program
        self.generator = p_generator
        self.source_ps = p_sourceps
        
        self.shell_list = self.source_ps.get_neighboring_shells()
        self.shell_dict = {
            self.shell_list[i]:i for i in range(len(self.shell_list))
        }
        
    def __len__(self):
        return self.n_program
    
    '''
    shortened method: only valid for size 1 programs
    '''
    def __getitem__(self, p_ind):
        d_solution = self.generator.get_new_chain_program(2)
        d_shell = d_solution.shells[0]
        dind_shell = self.shell_dict[d_shell]
        dabs_input = d_solution.interpreter.camb_get_ventogyrus(d_solution.inputs[0])
        return (np.array(dind_shell), np.array(dabs_input))

In [ ]:
class BLE(nn.Module):
    def __init__(self, p_config=None):
        super(BLE,self).__init__()
        self.config = p_config
        self.fc1 = nn.Linear(
            self.config["BLE"]["input_size"],
            2048,
        )
        self.fc2 = nn.Linear(
            2048,
            self.config["BLE"]["output_size"],
        )
    def forward(self, p_abs):
        # p_abs: (B, 15*7+1)
        out1 = F.relu(self.fc1(p_abs))
        out2 = torch.log_softmax(self.fc2(out1),dim=1)
        return out2
        

In [ ]:
def BLETrain(p_config, p_engine, pld_data, p_optim, p_lossfn):
    for d_ep in range(p_config["BLE"]["n_epoch"]):
        ep_loss_list = []
        for batch_idx, (b_ind, b_input) in enumerate(pld_data):
            p_engine.train()
            if use_cuda:
                tb_ind = Variable(b_ind).long().cuda()
                tb_input = Variable(b_input).float().cuda()
            else:
                tb_ind = Variable(b_ind).long()
                tb_input = Variable(b_input).float()
            
            tb_preds = p_engine(tb_input) # (B, output_size)
            tb_loss = p_lossfn(tb_preds, tb_ind)
            ep_loss_list.append(tb_loss)
            p_optim.zero_grad()
            tb_loss.backward()
            p_optim.step()
            
            print("\r# Train Ep:{}, B:{}, epLoss:{:.2f}".format(
                d_ep, batch_idx, sum(ep_loss_list)/len(ep_loss_list)
            ),end="")
            

In [ ]:
m_interpreter = MorpheusInterpreter()
m_spec = S.parse_file('./example/camb5.tyrell')
m_generator = MorpheusGenerator(m_spec, m_interpreter)
m_ps = ProgramSpace(
    m_spec, m_interpreter, [None], None
)

In [ ]:
m_config = {
    "BLE":{
        "ntrain_program":1000,
        "input_size": 15*7+1,
        "output_size": len(m_ps.get_neighboring_shells()),
        "n_epoch": 100,
    }
}

In [ ]:
dt_pd = ProgramDataset(
    p_config=m_config, 
    pn_program=m_config["BLE"]["ntrain_program"], 
    p_generator=m_generator, 
    p_sourceps=m_ps
)
ld_pd = DataLoader(dataset=dt_pd, batch_size=8, shuffle=True)

In [ ]:
ble = BLE(p_config=m_config)
m_lossfn = nn.NLLLoss()
m_optim = torch.optim.Adam(ble.parameters())
if use_cuda:
    ble = ble.cuda()
    m_lossfn = m_lossfn.cuda()

In [ ]:
BLETrain(m_config, ble, ld_pd, m_optim, m_lossfn)

# Train Ep:0, B:96, epLoss:4.61